In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.utils import shuffle, resample
from scipy.stats import chi2
from scipy.stats import norm
from scipy.stats import t
from tqdm import tqdm

In [2]:
sample = np.array([0.974,0.950,0.932,1.104,1.038,0.920,0.935,0.907,0.810,0.915])